In [ ]:
colab api

说明：
Google Drive API认证：该脚本首先通过Google Drive API认证，并建立Drive服务对象，用于文件的上传和下载。
上传到Google Drive：指定.ipynb文件的路径，将其上传到Google Drive。
手动运行Colab Notebook：目前，Google Colab并未提供自动化API来启动和运行notebook，因此，用户需要手动进入生成的URL来运行notebook。
下载结果文件：假定用户已经运行notebook并生成了一个结果文件，脚本会下载该文件到本地。
注意：
credentials.json文件是你从Google Cloud Platform获取的OAuth凭据文件。
你需要手动执行在Colab上运行的步骤，因为Google Colab不允许通过脚本完全自动化运行notebook的过程。

In [3]:
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
import os
import pickle
import io

# Authenticate and create the drive service
def authenticate_gdrive():
    SCOPES = ['https://www.googleapis.com/auth/drive']
    creds = None

    # Load credentials if available
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    # If no valid credentials, prompt user to log in
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)

        # Save credentials for future use
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    return build('drive', 'v3', credentials=creds)

# Upload a file to Google Drive
def upload_to_drive(service, file_path):
    file_metadata = {'name': os.path.basename(file_path)}
    media = MediaFileUpload(file_path, mimetype='application/octet-stream')
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    return file.get('id')

# Run notebook on Google Colab
def run_notebook_on_colab(file_id):
    colab_url = f'https://colab.research.google.com/drive/{file_id}'
    print(f'Run your notebook at: {colab_url}')
    # Colab does not have a public API to run notebooks automatically, manual interaction is required.

# Download the file from Google Drive
def download_from_drive(service, file_id, output_path):
    request = service.files().get_media(fileId=file_id)
    fh = io.FileIO(output_path, 'wb')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while not done:
        status, done = downloader.next_chunk()
        print(f'Download progress: {int(status.progress() * 100)}%')

if __name__ == '__main__':
    # Authenticate Google Drive
    drive_service = authenticate_gdrive()

    # Define the local file path
    file_path = 'path/to/your/notebook.ipynb'

    # Upload to Google Drive
    file_id = upload_to_drive(drive_service, file_path)
    print(f'File uploaded to Google Drive with ID: {file_id}')

    # Run the notebook on Google Colab (Manual Step)
    run_notebook_on_colab(file_id)

    # You need to manually run the notebook and then download the result.

    # Assume the output file ID (you'll need to get this from Colab after running the notebook)
    output_file_id = 'your-output-file-id'

    # Download the result to a local path
    output_path = 'path/to/save/output.ipynb'
    download_from_drive(drive_service, output_file_id, output_path)
    print(f'Result downloaded to {output_path}')


FileNotFoundError: [Errno 2] No such file or directory: 'credentials.json'

In [2]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/be/8a/fe34d2f3f9470a27b01c9e76226965863f153d5fbe276f83608562e49c04/google_auth_httplib2-0.2.0-py2.py3-none-any.whl (9.3 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 2.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 3.4 MB/s eta 0:00:00
